## Duplicate Checking Script

In [ ]:
from datetime import timedelta
import math
import numpy as np
import pandas as pd

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
plotly.offline.init_notebook_mode(connected=True)

import seaborn as sns
import matplotlib.pyplot as plt


import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

import matplotlib.pyplot as plt

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

In [ ]:
duplicate_data = data.copy()

#right, had to create a function to change the datetime data
def float_time_to_string(float_time):
    hours = int(float_time)
    decimal_part = float_time - hours

    minutes = int(decimal_part * 60)

    return f"{hours:02d}:{minutes:02d}"

duplicate_data['hora_creacion'] = duplicate_data['hora_creacion'].apply(float_time_to_string)


# 'fecha_creacion' and 'hora_creacion' to new column 'datetime' and sort by this column
duplicate_data['datetime'] = pd.to_datetime(duplicate_data['fecha_creacion'] + ' ' + duplicate_data['hora_creacion'])
duplicate_data = duplicate_data.sort_values(by='datetime')

In [ ]:
# check if it's geographically close
def is_close_enough(lat1, lon1, lat2, lon2):
    threshold = 0.001  # something close to about 100 meters for latitude
    distance_lat = abs(lat1 - lat2)
    distance_lon = abs(lon1 - lon2) * math.cos(math.radians((lat1 + lat2) / 2))
    return distance_lat < threshold and distance_lon < threshold


In [ ]:
# duplicate function, goes through 
def is_duplicate(row, previous_rows):
    for prev_row in previous_rows.itertuples():
        # check fifteen minutes
        if row.datetime - prev_row.datetime > timedelta(minutes=10):
            continue

        # check alcaldia
        if row.alcaldia_cierre != prev_row.alcaldia_cierre:
            continue

        # check tipo incident
        if row.tipo_incidente_c4 != prev_row.tipo_incidente_c4:
            continue

        # use proximity function to check how close
        if not is_close_enough(row.latitud, row.longitud, prev_row.latitud, prev_row.longitud):
            continue

        return True  # it's a duplicate (by the standard of our function, we can add more)
    return False


In [ ]:
# add new column to show duplicate
duplicate_data['is_duplicate'] = False
for index, row in duplicate_data.iterrows():
    # limit rows to the last 100
    previous_rows = duplicate_data[duplicate_data['datetime'] < row.datetime].tail(100)
    duplicate_data.at[index, 'is_duplicate'] = is_duplicate(row, previous_rows)


In [ ]:
duplicate_data.tail(50)


In [ ]:
duplicate_data.to_csv('data911_dupeCheck.csv', index = False)